In [1]:
%cd .. 

/home/d/PycharmProjects/protein_properties


In [2]:
# For automatic file reloading as usual
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
from data.lightning import SASADataConfig, SASADataModule, SASADataset
from models import SASABaseline
import os 
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

In [7]:
config = SASADataConfig(data_dir="data/sasa", embedding_path="data/sasa/sasaembeddings.h5", num_workers=4, num_classes=2, np_path="data/sasa/")
data_module = SASADataModule(config=config)
data_module.setup("train")

Data preparation already done!


### Overfit on one sample -> check that the model is working and the data makes sense

In [4]:
logger = TensorBoardLogger('lightning_logs', name='sasa_overfit_one_sample')
one_sample_train_set = DataLoader(data_module.train_dataset[42], batch_size=1, shuffle=False, num_workers=4)
one_samples_trainer = pl.Trainer(accelerator="gpu", devices=1, deterministic=True, max_epochs=5, enable_checkpointing=False, limit_train_batches=1, profiler="advanced", logger=logger)
model = SASABaseline(config.num_classes)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

In [ ]:
one_samples_trainer.fit(model, one_sample_train_set, data_module.validation_dataloader())

### Now try some more samples

In [ ]:
logger = TensorBoardLogger('lightning_logs', name='sasa_overfit')
one_sample_train_set = DataLoader(data_module.train_dataset[torch.randperm(len(data_module.train_dataset))[:10]], batch_size=1, shuffle=True, num_workers=4)
one_samples_trainer = pl.Trainer(accelerator="gpu", devices=1, deterministic=True, max_epochs=10, enable_checkpointing=False, accumulate_grad_batches=10, profiler="advanced", logger=logger)
model = SASABaseline(config.num_classes)